In [56]:
from __future__ import annotations
import zipfile
import pandas as pd
import os
from io import StringIO
import matplotlib.pyplot as plt
import tempfile
import plotly.express as px


In [57]:
DATA_ZIP = "/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/ALL/windtunnel_data.zip"
OUTPUT_DIR = "/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/ALL/"

In [58]:
df = pd.DataFrame(columns=["Motor", "Prop", "Airspeed", "Cells", "Battery", "Config", "Thrust"])

with zipfile.ZipFile(DATA_ZIP) as z:
    for name in filter(lambda x: x.endswith(".csv") and not x.startswith('__MACOSX'), z.namelist()):
        motor, prop, airspeed, cells, battery, config = name.removeprefix("windtunnel_data/").removesuffix(".csv").split("_")
        data = z.read(name)
        _df = pd.read_csv(StringIO(data.decode('utf-8')))
        print(name)
        if config != "PT":
            _df = _df[_df["CODE"] == 0]
            _df.reset_index(inplace=True, drop=True)
        thrust = _df["Drag"].min() * -1
        
        df.loc[-1] = [motor, prop.lower(), airspeed, cells, battery, config, thrust]
        df.index = df.index + 1

windtunnel_data/AT7215_16x14_30ms_12S_NaN_PT.csv
windtunnel_data/AT7215_15x14x3_45ms_12S_115Wh_M2.csv
windtunnel_data/AT7215_15x14x3_30ms_12S_97.5Wh_M3.csv
windtunnel_data/AT7215_16x14_45ms_12S_NaN_MAX.csv
windtunnel_data/AT5330_16x14_20ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_45ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_20ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_10ms_12S_115Wh_RUN1.csv
windtunnel_data/AT5220_15X14X3_40ms_8S_2.4aH_RUN1.csv
windtunnel_data/AT5330_16x16_30ms_12S_NaN_PT.csv
windtunnel_data/AT7215_15x14x3_35ms_12S_97.5Wh_M3.csv
windtunnel_data/AT5220_16X12_30ms_8S_NaN_RUN1.csv
windtunnel_data/AT7215_16x14_15ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x16_45ms_12S_NaN_PT.csv
windtunnel_data/AT5220_15X14X3_35ms_8S_2.0aH_RUN1.csv
windtunnel_data/AT7215_16x14_40ms_12S_97.5Wh_M3.csv
windtunnel_data/AT5330_16x14_30ms_12S_NaN_PT.csv
windtunnel_data/AT5220_16X12_35ms_8S_NaN_RUN1.csv
windtunnel_data/AT7215_16x14_10ms_12S_NaN_PT.csv
windtunnel_data/AT7215_16x14_static

In [59]:
df = df[df["Airspeed"] != "NaN"]

In [60]:
vcounts = df.value_counts()
vcounts.to_csv(os.path.join(OUTPUT_DIR, "vcounts.csv"))

In [61]:
# find rows with duplicated "Motor", "Prop", "Airspeed", "Cells" and print the battery, config and thrust
duplicated = df[df.duplicated(subset=["Motor", "Prop", "Airspeed", "Cells"], keep=False)]
duplicated[["Motor", "Prop", "Airspeed", "Cells", "Battery", "Config", "Thrust"]].groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg(list)

Battery            Config  \
Motor  Prop    Airspeed Cells                                           
AT5220 15x14x3 35ms     8S           [2.0aH, 3.2aH]      [RUN1, RUN1]   
AT7215 15x14x3 10ms     12S           [97.5Wh, NaN]        [RUN1, PT]   
               30ms     12S           [97.5Wh, NaN]          [M3, PT]   
               35ms     12S           [97.5Wh, NaN]          [M3, PT]   
               40ms     12S    [115Wh, 97.5Wh, NaN]      [M2, M3, PT]   
               45ms     12S            [115Wh, NaN]          [M2, PT]   
               static   12S           [NaN, 97.5Wh]        [PT, RUN1]   
       16x14   10ms     12S            [115Wh, NaN]        [RUN1, PT]   
               30ms     12S           [NaN, 97.5Wh]          [PT, M3]   
               35ms     12S           [NaN, 97.5Wh]          [PT, M3]   
               40ms     12S    [97.5Wh, NaN, 115Wh]      [M3, PT, M2]   
               45ms     12S       [NaN, NaN, 115Wh]     [MAX, PT, M2]   
               50ms     12S              [NaN, NaN]         [PT, MAX]   
               static   12S    [115Wh, NaN, 97.5Wh]  [RUN1, PT, RUN1]   

                                                 Thrust  
Motor  Prop    Airspeed Cells                            
AT5220 15x14x3 35ms     8S                [13.1, 14.77]  
AT7215 15x14x3 10ms     12S             [23.37, 27.702]  
               30ms     12S             [7.195, 21.683]  
               35ms     12S             [8.039, 20.938]  
               40ms     12S     [13.304, 6.403, 18.317]  
               45ms     12S             [12.82, 16.533]  
               static   12S             [26.239, 23.61]  
       16x14   10ms     12S            [17.386, 24.399]  
               30ms     12S             [23.756, 6.404]  
               35ms     12S             [23.357, 6.225]  
               40ms     12S     [6.281, 21.648, 13.901]  
               45ms     12S    [17.585, 21.368, 14.432]  
               50ms     12S            [18.339, 16.346]  
               static   12S    [18.291, 25.753, 20.616]

In [62]:
duplicated[duplicated["Battery"] != "NaN"].groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg(list)

Battery        Config            Thrust
Motor  Prop    Airspeed Cells                                                 
AT5220 15x14x3 35ms     8S      [2.0aH, 3.2aH]  [RUN1, RUN1]     [13.1, 14.77]
AT7215 15x14x3 10ms     12S           [97.5Wh]        [RUN1]           [23.37]
               30ms     12S           [97.5Wh]          [M3]           [7.195]
               35ms     12S           [97.5Wh]          [M3]           [8.039]
               40ms     12S    [115Wh, 97.5Wh]      [M2, M3]   [13.304, 6.403]
               45ms     12S            [115Wh]          [M2]           [12.82]
               static   12S           [97.5Wh]        [RUN1]           [23.61]
       16x14   10ms     12S            [115Wh]        [RUN1]          [17.386]
               30ms     12S           [97.5Wh]          [M3]           [6.404]
               35ms     12S           [97.5Wh]          [M3]           [6.225]
               40ms     12S    [97.5Wh, 115Wh]      [M3, M2]   [6.281, 13.901]
               45ms     12S            [115Wh]          [M2]          [14.432]
               static   12S    [115Wh, 97.5Wh]  [RUN1, RUN1]  [18.291, 20.616]

In [63]:
df[df["Config"] == "M3"].sort_values(by=["Motor", "Prop", "Airspeed", "Cells"]).reset_index(drop=True)

,Motor,Prop,Airspeed,Cells,Battery,Config,Thrust
0,AT7215,15x14x3,30ms,12S,97.5Wh,M3,7.195
1,AT7215,15x14x3,35ms,12S,97.5Wh,M3,8.039
2,AT7215,15x14x3,40ms,12S,97.5Wh,M3,6.403
3,AT7215,16x14,30ms,12S,97.5Wh,M3,6.404
4,AT7215,16x14,35ms,12S,97.5Wh,M3,6.225
5,AT7215,16x14,40ms,12S,97.5Wh,M3,6.281


In [64]:
df[df["Config"] == "M2"].sort_values(by=["Motor", "Prop", "Airspeed", "Cells"]).reset_index(drop=True)

,Motor,Prop,Airspeed,Cells,Battery,Config,Thrust
0,AT7215,15x14x3,40ms,12S,115Wh,M2,13.304
1,AT7215,15x14x3,45ms,12S,115Wh,M2,12.820
2,AT7215,16x14,40ms,12S,115Wh,M2,13.901
3,AT7215,16x14,45ms,12S,115Wh,M2,14.432


In [65]:
# keep only the max thrust for each "Motor", "Prop", "Airspeed", "Cells"
df_max = df.groupby(["Motor", "Prop", "Airspeed", "Cells"]).agg({"Thrust": "max"}).reset_index()
df_max

,Motor,Prop,Airspeed,Cells,Thrust
0,AT5220,15x14x3,25ms,8S,15.538
1,AT5220,15x14x3,30ms,8S,14.885
2,AT5220,15x14x3,35ms,8S,14.770
3,AT5220,15x14x3,40ms,8S,10.570
4,AT5220,15x14x3,static,8S,18.135
5,AT5220,16x12,25ms,8S,15.762
6,AT5220,16x12,30ms,8S,14.680
7,AT5220,16x12,35ms,8S,13.679
8,AT5220,16x12,40ms,8S,13.343
9,AT5220,16x12,static,8S,16.754


In [66]:
df_max["Airspeed"] = df_max["Airspeed"].apply(lambda x: float(x.replace("static", "0.0").replace("ms", "")))

In [67]:
df_max_8s = df_max[df_max["Cells"] == "8S"]
df_max_12s = df_max[df_max["Cells"] == "12S"]

In [68]:
for motor in df_max_8s["Motor"].unique():
    df_motor = df_max_8s[df_max_8s["Motor"] == motor].sort_values(by="Airspeed")
    fig = px.scatter(df_motor, x="Airspeed", y="Thrust", color="Prop", title=motor + " 8S")
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        xaxis_title="Airspeed (m/s)",
        yaxis_title="Thrust (lbf)",
        legend_title="Prop",
    )
    fig.add_vrect(
        x0=35, x1=55,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(55 + 35)/2, y=df_motor["Thrust"].max(),
        text="M2",
        showarrow=False,
        font=dict(
            size=16,
            color="red"
        ),
    )
    fig.add_vrect(
        x0=20, x1=30,
        fillcolor="LightSeagreen", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(30 + 20)/2, y=df_motor["Thrust"].max(),
        text="M3",
        showarrow=False,
        font=dict(
            size=16,
            color="seagreen"
        ),
    )
    fig.show()
    fig.write_image(os.path.join(OUTPUT_DIR, f"{motor}_8s.png"), width=1000, height=500)

In [69]:
for motor in df_max_12s["Motor"].unique():
    df_motor = df_max_12s[df_max_12s["Motor"] == motor].sort_values(by="Airspeed")
    fig = px.scatter(df_motor, x="Airspeed", y="Thrust", color="Prop", title=motor + " 12S")
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        xaxis_title="Airspeed (m/s)",
        yaxis_title="Thrust (lbf)",
        legend_title="Prop",
    )
    fig.add_vrect(
        x0=35, x1=55,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(55 + 35)/2, y=df_motor["Thrust"].max(),
        text="M2",
        showarrow=False,
        font=dict(
            size=16,
            color="red"
        ),
    )
    fig.add_vrect(
        x0=20, x1=30,
        fillcolor="LightSeagreen", opacity=0.5,
        layer="below", line_width=0,
    )
    fig.add_annotation(
        x=(30 + 20)/2, y=df_motor["Thrust"].max(),
        text="M3",
        showarrow=False,
        font=dict(
            size=16,
            color="seagreen"
        ),
    )
    fig.show()
    fig.write_image(os.path.join(OUTPUT_DIR, f"{motor}_12s.png"), width=1000, height=500)